In [1]:
import time
import pandas as pd
import dask.dataframe as dd
import modin.pandas as mpd
import yaml
import os

In [2]:
file_path = "Large-file.csv"

# Method 1: Pandas
start_time = time.time()
df_pandas = pd.read_csv(file_path)
end_time = time.time()
time_pandas = end_time - start_time
print("Time taken by pandas: ", time_pandas, " seconds")

# Method 2: Dask
start_time = time.time()
df_dask = dd.read_csv(file_path)
df_dask.compute()
end_time = time.time()
time_dask = end_time - start_time
print("Time taken by dask: ", time_dask, " seconds")

# Method 3: Modin
# start_time = time.time()
# df_modin = mpd.read_csv(file_path)
# end_time = time.time()
# time_modin = end_time = start_time
# print("Time taken by Modin: ", time_modin, " seconds") 

# NOTE ====> MODIN IS BROKE

Time taken by pandas:  0.0066950321197509766  seconds
Time taken by dask:  0.013063907623291016  seconds


In [4]:
# Perform basic validation on data coluns

df = pd.read_csv(file_path)

# Remove special characters from column names
df.columns = df.columns.str.replace('[^a-zA-Z0-9]+', '_')

# Remove whitespaces from column names
df.columns = df.columns.str.strip().str.lower()

# Check for missing values in each column
missing_values = df.isnull().sum()

# Check for duplicates
duplicates = df[df.duplicated()]

In [5]:
# Get all the columns
all_cols = df.columns.tolist()

# Print the columns
print(all_cols)

['index', 'ean', 'internal id', 'stock', 'price']


In [9]:
# Validating number of columns and column name of ingested file with YAML

import yaml

with open('data_types.yaml', 'r') as f:
    schema = yaml.safe_load(f)

if 'columns' not in schema:
    print("Error: Invalid schema file :( 'columns' key is missing.")
elif not isinstance(schema['columns'], list):
    print("Error: Invalid schema file. 'columns' values should be a list")
else: 
    if len(df.columns) != len(schema['columns']):
        print("Error: Number of columns in the file does not match the schema.")
    else:
        for i, col in enumerate(df.columns):
            if col != schema['columns'][i]:
                print(f"Error: Column {i + 1} doesn't match the schema.")

Error: Invalid schema file. 'columns' values should be a list


In [10]:
subset_df = df.iloc[:1000] # Selecting first 1000 rows for less complexity
subset_df.to_csv('output_file.csv.gz', sep = '|', compression = 'gzip', index = False)

In [11]:
# Get the file size
file_size = os.path.getsize('output_file.csv.gz')

num_rows = len(df)
num_cols = len(df.columns)

print(f"Total number of rows: {num_rows}")
print(f"Total number of columns: {num_cols}")
print(f"File size: {file_size} bytes")

Total number of rows: 5000
Total number of columns: 5
File size: 14792 bytes
